In [1]:
import numpy as np
import lime
import sklearn
import sklearn.ensemble
import numpy as np
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np

data = np.genfromtxt('../datasets/titanic.txt', delimiter=',')
y_train = data[:, -1]
X_train = data[:, :-1]


In [3]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from Anchor.anchor import Anchor, Tasktype

c = sklearn.ensemble.RandomForestClassifier(n_estimators=100, n_jobs=5, random_state=123)
c.fit(X_train, y_train)
print('Train', sklearn.metrics.accuracy_score(y_train, c.predict(X_train)))

explainer = Anchor(Tasktype.TABULAR)

task_paras = {"dataset": X_train, "column_names": ["PcClass", "Name", "Sex", "Age", "SibSp", "Parch", "Ticket", "Fare", "Cabin", "Embarked"]}
method_paras = {"beam_size": 1, "desired_confidence": 1.0}
anchor = explainer.explain_instance(
    input=X_train[759].reshape(1, -1),
    predict_fn=c.predict,
    method="beam",
    task_specific=task_paras,
    method_specific=method_paras,
    num_coverage_samples=100,
)

INFO:root: Start Sampling
INFO:root: Start Beam Search


Train 1.0
eps 0.15
while loop 0.9999977052211761
while loop 0.9326510280370712
while loop 0.9718704614788294
while loop 0.8528023348855043
while loop 0.7678715827327369
while loop 0.7164411779089231
while loop 0.6814010518550188
while loop 0.6555929258231401
while loop 0.6154776124269816
while loop 0.570395396169485
while loop 0.5392113419804956
while loop 0.5316683633522535
while loop 0.4888427021794677
while loop 0.4578831743793758
while loop 0.41870573842436754
while loop 0.38721905584633665
while loop 0.3710118957844313
while loop 0.35739375194222334
while loop 0.3393678432294178
while loop 0.3481742795248963
while loop 0.3373978613630262
while loop 0.32630102135214756
while loop 0.3088763702526939
while loop 0.3025558986773029
while loop 0.2718356883773362
while loop 0.26347364340825674
while loop 0.2708969737157273
while loop 0.2684829131651868
while loop 0.25791421811629955
while loop 0.25309398230921476
while loop 0.24416217529435824
while loop 0.21111028740605986
while loop 0.

In [6]:
anchor

AnchorCandidate(feature_mask=[2, 0], precision=1.0, n_samples=96, positive_samples=96, coverage=0.54)

In [7]:
visu = explainer.visualize(anchor, X_train[759])
print(visu)

PcClass = 1.0 AND Sex = 1.0
